In [3]:
import casadi as cs
from urdf_parser_py.urdf import URDF, Pose
import os # For current directory
import urdf2casadi.urdf2casadi.urdfparser
import urdf2casadi.urdf2casadi.geometry.plucker as plucker
import numpy as np
import PyKDL as kdl
import kdl_parser.kdl_parser_py.kdl_parser_py.urdf as kdlurdf
from timeit import Timer, timeit, repeat
import rbdl

ok, ur_tree = kdlurdf.treeFromFile('ur5_rbdl.urdf')
asd = urdf2casadi.urdf2casadi.urdfparser.URDFparser()
robot_desc = asd.from_file("ur5_rbdl.urdf")
root = 'base_link'
tip = 'wrist_3_link'
ur_chain = ur_tree.getChain(root,tip)
rbdlmodel = rbdl.loadModel("ur5_rbdl.urdf")

The root link base_link has an inertia specified in the URDF, but KDL does not support a root link with an inertia.  As a workaround, you can add an extra dummy link to your URDF.
ur5_rbdl.urdf


In [4]:
jlist, names, qmax, qmin = asd.get_joint_info(root, tip)
njoints = len(jlist)
print njoints
print len(qmax)

H_ur = kdl.JntSpaceInertiaMatrix(6)

grav = kdl.Vector()
grav[2] = -9.81

q = kdl.JntArray(njoints)
qdot = kdl.JntArray(njoints)

q_np = np.zeros(njoints)
qdot_np = np.zeros(njoints)
qddot_np = np.zeros(njoints)

q_sx = cs.SX.zeros(njoints)
print q_sx
qdot_sx = cs.SX.zeros(njoints)

q_none = [None]*njoints
qdot_none = [None]*njoints
qddot_none = [None]*njoints

tau_kdl = kdl.JntArray(njoints)
grav_kdl = kdl.JntArray(njoints)

nitr = 50000
nrepeat = 10

6
6
@1=0, [@1, @1, @1, @1, @1, @1]


In [ ]:
g = asd.get_gravity_RNEA(root, tip, grav)
def g_u2c_func():    
    for j in range(njoints):
        q_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        #qdot_none[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    g(q_none)

#timeit_g_u2c = timeit("g_u2c_func()", setup = "from __main__ import g_u2c_func", number = nitr)
#print timeit_g_u2c
#print timeit_g_u2c/nitr
timeit_g_u2c_repeat = repeat("g_u2c_func()", setup = "from __main__ import g_u2c_func", repeat = 10, number = nitr)
print timeit_g_u2c_repeat


#print g(np.ones(njoints))
#for i in range(njoints):
#    q[i] = 1.
#kdl.ChainDynParam(ur_chain, grav).JntToGravity(q, tau_kdl)
#print tau_kdl

In [ ]:
def g_kdl():
    for j in range(njoints):
        q[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2
        #qdot[j] = (qmax[j] - qmin[j])*np.random.rand()-(qmax[j] - qmin[j])/2

    kdl.ChainDynParam(ur_chain, grav).JntToGravity(q, tau_kdl)
#kdl_time = [None]*30

#timeit_g_kdl = timeit("g_kdl()", setup = "from __main__ import g_kdl", number = nitr)
#print timeit_g_kdl
#print timeit_g_kdl/nitr
timeit_g_kdl_repeat = repeat("g_kdl()", setup = "from __main__ import g_kdl", repeat = nrepeat, number = nitr)
#print timeit_g_kdl_repeat
